In [1]:
!wget --no-check-certificate \
https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip -O /tmp/rockpaperscissors.zip

--2020-05-04 08:48:46--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  22.8MB/s    in 13s     

2020-05-04 08:48:59 (23.2 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



In [2]:
from zipfile import ZipFile
import os

local_zip = '/tmp/rockpaperscissors.zip'

zip_ref = ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/rockpaperscissors'

print('paper:',len(os.listdir(os.path.join(base_dir, 'paper'))))
print('rock:',len(os.listdir(os.path.join(base_dir, 'rock'))))
print('scissors:',len(os.listdir(os.path.join(base_dir, 'scissors'))))

paper: 712
rock: 726
scissors: 750


Modify the file structure

In [0]:
!rm -rf /tmp/rockpaperscissors/rps-cv-images/
!rm /tmp/rockpaperscissors/README_rpc-cv-images.txt

Split training & test

In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd

training_files = []
training_labels = []
validation_files = []
validation_labels = []

for label in os.listdir(base_dir):
    # label for each file
    labels = []
    # file path
    files = []

    for item in os.listdir(os.path.join(base_dir, label)):
        labels.append(label)
        files.append(os.path.join(base_dir, label, item))
    
    train_file, validation_file, train_label, validation_label = train_test_split(files, labels, train_size=500)
    training_files += train_file
    training_labels += train_label
    validation_files += validation_file
    validation_labels += validation_label

training_df = pd.DataFrame({'labels': training_labels, 'filepath': training_files})
validation_df = pd.DataFrame({'labels': validation_labels, 'filepath': validation_files})

Create DataImageGenerator

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)

test_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)

Prepare training data

In [6]:
train_generator = train_data_gen.flow_from_dataframe(
    training_df,
    class_mode='categorical',
    x_col='filepath',
    y_col='labels',
    target_size=(150,150),
    batch_size=4
)

val_generator = test_data_gen.flow_from_dataframe(
    validation_df,
    class_mode='categorical',
    x_col='filepath',
    y_col='labels',
    target_size=(150,150),
    batch_size=4
)

Found 1500 validated image filenames belonging to 3 classes.
Found 688 validated image filenames belonging to 3 classes.


Build CNN

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.Adam(),
    metrics=['accuracy']
)

model.fit(
    train_generator,
    steps_per_epoch=25,
    epochs=20,
    validation_data=val_generator,
    validation_steps=5,
    verbose=2
)

Epoch 1/20
25/25 - 1s - loss: 1.1077 - accuracy: 0.3300 - val_loss: 1.0324 - val_accuracy: 0.5500
Epoch 2/20
25/25 - 1s - loss: 1.0963 - accuracy: 0.4700 - val_loss: 1.0196 - val_accuracy: 0.3500
Epoch 3/20
25/25 - 1s - loss: 0.9170 - accuracy: 0.6100 - val_loss: 0.9924 - val_accuracy: 0.4500
Epoch 4/20
25/25 - 1s - loss: 0.7024 - accuracy: 0.6600 - val_loss: 0.7408 - val_accuracy: 0.5500
Epoch 5/20
25/25 - 1s - loss: 0.6228 - accuracy: 0.7900 - val_loss: 0.4790 - val_accuracy: 0.8000
Epoch 6/20
25/25 - 1s - loss: 0.4105 - accuracy: 0.8100 - val_loss: 0.5848 - val_accuracy: 0.7500
Epoch 7/20
25/25 - 1s - loss: 0.3370 - accuracy: 0.8900 - val_loss: 0.4558 - val_accuracy: 0.8000
Epoch 8/20
25/25 - 1s - loss: 0.2944 - accuracy: 0.8900 - val_loss: 0.6526 - val_accuracy: 0.8000
Epoch 9/20
25/25 - 1s - loss: 0.3064 - accuracy: 0.9100 - val_loss: 0.4520 - val_accuracy: 0.8500
Epoch 10/20
25/25 - 1s - loss: 0.4543 - accuracy: 0.8200 - val_loss: 0.3558 - val_accuracy: 0.8000
Epoch 11/20
25/25 -